In [ ]:
!git clone https://github.com/ManuelZ/image-search-engine.git

In [ ]:
!wget --no-clobber <<FILL ME>>
!wget --no-clobber <<FILL ME>>
!unzip -nq <<FILL ME>>.zip
!unzip -nq <<FILL ME>>.zip

In [ ]:
!pip install --quiet --upgrade pip

### Change directory to be able to make the imports

In [ ]:
import os
os.chdir(r'/<<FILL ME>>/image-search-engine/backend')

In [ ]:
# External imports
import tensorflow as tf
import cv2

# Local imports
import siamese.config as config
from siamese.dataset import CommonMapFunction
from siamese.create_index import create_faiss_index, create_manual_index, create_one_head_net
from siamese.test_index import read_index, query_index, display_query_results

In [ ]:
config.OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
!pip install --quiet -r siamese/requirements.txt

In [ ]:
!python -m siamese.train

### Create index

In [ ]:
model_path = config.LOAD_MODEL_PATH
data_path = config.DATA_SUBSET

if config.INDEX_TYPE == "faiss":
    create_faiss_index(model_path, data_path)
elif config.INDEX_TYPE == "dict":
    create_manual_index(model_path, data_path)

### Test index

In [ ]:
one_head_net = create_one_head_net(config.LOAD_MODEL_PATH)
map_fun = CommonMapFunction(config.IMAGE_SIZE)

query_paths = list(config.QUERY_DATASET.rglob("**/*.[jp][pn]g"))
print(f"There are {len(query_paths)} images for querying.")

index = read_index()

for query_path in query_paths:

    # Load and preprocess image
    image = map_fun.decode_and_resize(str(query_path))

    # Add batch dimension
    image = tf.expand_dims(image, 0, name=None)

    # Extract embeddings
    embedding = one_head_net(image).numpy()

    indices, distances = query_index(
        embedding, index, config.INDEX_TYPE, config.N_RESULTS
    )

    # Display query and results
    im_query = cv2.imread(str(query_path))
    im_query = cv2.cvtColor(im_query, cv2.COLOR_BGR2RGB)
    display_query_results(im_query, distances, indices)